# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [3]:
# Libraries
# import libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [4]:
# your code here
db = pd.read_csv ("Admission_Predict.csv")
db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


In [4]:
db.shape

(385, 9)

Remove trailing spaces at the end of the column names if there are any.


In [5]:
# your code here
db.columns = db.columns.str.strip()
db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Let's evaluate the dataset by looking at the `head` function.

In [6]:
# your code here
db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [6]:
# your code here
db.isnull()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
380,False,False,False,False,False,False,False,False,False
381,False,False,False,False,False,False,False,False,False
382,False,False,False,False,False,False,False,False,False
383,False,False,False,False,False,False,False,False,False


In [7]:
db.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [7]:
# your code here
db.head()



,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


In [8]:
db = db.set_index('Serial No.')             

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [10]:
# your code here
print(len(db["CGPA"].unique()))

168


In [11]:
print(len(db["GRE Score"].unique()))

49


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [15]:
# your code here

db.set_index(['GRE Score', 'CGPA'], inplace=True)
db.head()


,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90


In [11]:
db.drop(['GRE Score', 'CGPA'], axis=1, inplace=True)
db.head()

,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,1,118,4,4.5,4.5,1,0.92
1,2,104,3,3.0,3.5,1,0.72
2,3,110,3,3.5,2.5,1,0.80
3,4,103,2,2.0,3.0,0,0.65
4,5,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [13]:
# your code here

db[(db["CGPA"] > 9) & (db["Research"]==1)]

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
29,338,118,4,3.0,4.5,9.40,1,0.91
141,326,114,3,3.0,3.0,9.11,1,0.83
218,324,111,4,3.0,3.0,9.01,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [15]:
# your code here

db = db[(db['CGPA'] > 9) & (db['SOP'] < 3.5)]
db

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
29,338,118,4,3.0,4.5,9.40,1,0.91
63,327,114,3,3.0,3.0,9.02,0,0.61
141,326,114,3,3.0,3.0,9.11,1,0.83
218,324,111,4,3.0,3.0,9.01,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84


In [26]:
mean_count2 = count2.mean()
print(mean_count2)

Serial No.           166.600
GRE Score            328.000
TOEFL Score          112.800
University Rating      3.400
SOP                    3.000
LOR                    3.400
CGPA                   9.130
Research               0.800
Chance of Admit        0.802
dtype: float64


In [9]:
mean_chance = db[(db["CGPA"] > 9) & (db["SOP"]<3.5)]
mean_chance["Chance of Admit"].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [16]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    
    mean = np.mean(col)
    std = np.std(col)
    return (col - mean) / std
    
db['A_standardized'] = standardize(db['A'])
db['B_standardized'] = standardize(db['B'])





KeyError: 'A'

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [19]:
# your code here
db['CGPA_std'] = standardize_column(db['CGPA'])
db['GRE_std'] = standardize_column(db['GRE Score'])
db['LOR_std'] = standardize_column(db['LOR'])

NameError: name 'standardize_column' is not defined

We will generate the decision choice at random using the code below. Please run the cell.

In [17]:
# Libraries
from random import choices

In [ ]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [ ]:
# your code here

x = list(db["deciding_column"])

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [ ]:
# your code here

decision = []
for item in x:
    if item > 0.8:
        decision.append(1)
    else:
        decision.append(0)
admissions["decision"] = decision

admissions

How many applicants will be accepted to the program using the decision column? Compute the result below.

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
# your code here

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
# your code here